# Lunar Lander - Revised

Inspiration for this notebook comes from this [Keras blog post](https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html), the [VGG ConvNet paper](https://arxiv.org/pdf/1409.1556.pdf), and the [OpenAI Gym](https://gym.openai.com/). 

Train a model to play the Lunar Lander game from OpenAI gym using a convolutional neural network.

## Imports

In [ ]:
import os 
import cv2
import random
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline 
from matplotlib import ticker
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn import metrics
from imblearn.under_sampling import RandomUnderSampler
from sklearn.utils import shuffle

import keras
from keras.models import Sequential
from keras.layers import Input, Dropout, Flatten, Conv2D, MaxPooling2D, Dense, Activation
from keras.optimizers import RMSprop, adam
from keras.utils import np_utils

## Preparing the Data
Load the raw image dataset and pre-process it. Divide this into train and test splits. Balance the training split.

### Load Images

Loads a set of images and resizes each image to 64x64 and coverts to black and white. 

In [ ]:
# Set up some parmaeters for data loading
TRAIN_DIR = './data/'
sample_rate = 0.1
ROWS = 64
COLS = 64
CHANNELS = 1

# generate filenames from the data folder and do sampling
image_filenames = [TRAIN_DIR+i for i in os.listdir(TRAIN_DIR) if not i.startswith('.')] # use this for full dataset
image_filenames = random.sample(image_filenames, int(len(image_filenames)*sample_rate))

# Create a data array for image data
count = len(image_filenames)
data = np.ndarray((count, CHANNELS, ROWS, COLS), dtype=np.float)

# Iterate throuigh the filenames and for each one load the image, resize and normalise
for i, image_file in enumerate(image_filenames):
    image = cv2.imread(image_file, cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, (ROWS, COLS), interpolation=cv2.INTER_CUBIC)        
    data[i] = image
    data[i] = data[i]/255
    if i%1000 == 0: print('Processed {} of {}'.format(i, count))

print("Train shape: {}".format(data.shape))

### Generating the Labels

We're dealing with a multi-class classification problem here - (0) no action, (1) left, (2) up, and (3) right. The lables can be created by looping over the file names in the train directory.

In [ ]:
# Extract the lables from the last characters in the filename
labels = []
for i in image_filenames:
    l = i[-6:-5]
    labels.append(int(l))
        
# Count the number of clases
num_classes = len(set(labels))

# convert to binary encoded labels
labels_wide = keras.utils.to_categorical(labels, num_classes)

# Plot a bar plot of the 
sns.countplot(labels)

### Show some screens
Print a few screens with their labels.

In [ ]:
for i in range(0,5):
    idx = random.randint(0, len(labels))
    print(labels[idx])
    plt.figure(figsize=(3,3))
    plt.imshow(data[idx][0], cmap='gray')
    plt.show()

### Partition the dataset for evaluation
Split the data into a training and test partition so we can evaluate at the end

In [ ]:
train, test, train_labels, test_labels = train_test_split(data, labels, random_state=0, test_size = 0.2, train_size = 0.8)
train_labels_wide = keras.utils.to_categorical(train_labels, num_classes)
test_labels_wide = keras.utils.to_categorical(test_labels, num_classes)

Apply under sampling to balance the training dataset

In [ ]:
# Apply the random under-sampling
rus = RandomUnderSampler(return_indices=True)
train_rus, train_labels_rus, idx_resampled = rus.fit_sample(train.reshape(len(train), ROWS*COLS*CHANNELS), train_labels)
train_rus, train_labels_rus = shuffle(train_rus, train_labels_rus)
train_rus = train_rus.reshape(len(train_rus), CHANNELS,ROWS, COLS)
sns.countplot(train_labels_rus)
train_labels_rus_wide = keras.utils.to_categorical(train_labels_rus, num_classes)

### Show some screens
Print a few screens with their labels.

Data

In [ ]:
for i in range(0,5):
    idx = random.randint(0, len(labels))
    print(labels[idx])
    plt.figure(figsize=(3,3))
    plt.imshow(data[idx][0], cmap='gray')
    plt.show()

Train

In [ ]:
for i in range(0,5):
    idx = random.randint(0, len(train_labels))
    print(train_labels[idx])
    plt.figure(figsize=(3,3))
    plt.imshow(train[idx][0], cmap='gray')
    plt.show()

Train resampled

In [ ]:
for i in range(0,5):
    idx = random.randint(0, len(train_labels_rus))
    print(train_labels_rus[idx])
    plt.figure(figsize=(3,3))
    plt.imshow(train_rus[idx][0], cmap='gray')
    plt.show()

Test

In [ ]:
for i in range(0,5):
    idx = random.randint(0, len(test_labels))
    print(test_labels[idx])
    plt.figure(figsize=(3,3))
    plt.imshow(test[idx][0], cmap='gray')
    plt.show()

## LunarLanderNet

A simple convolutional network to control the lander.

In [ ]:
model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same', input_shape=(CHANNELS, ROWS, COLS), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(num_classes, activation = 'softmax'))

model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=adam(lr=1e-3), metrics=['accuracy'])

### Train
Train a model.

In [ ]:
epochs = 50
batch_size = 128

history = model.fit(train_rus, train_labels_rus_wide, batch_size=batch_size, epochs=epochs, validation_split=0.25, verbose=1, shuffle=True)

Plot the evolution of the loss as the module was trained.

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']

plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.plot(loss, 'blue', label='Training Loss')
plt.plot(val_loss, 'green', label='Validation Loss')
plt.xticks(range(0,epochs)[0::2])
plt.legend()
plt.show()

### Evaluate the Model
Use the test dataset to evaluate the model

In [ ]:
print("****** Test Data ********")
# Make a set of predictions for the validation data
pred = model.predict_classes(test)

# Print performance details
print(metrics.classification_report(test_labels, pred))

# Print confusion matrix
print("Confusion Matrix")
print(metrics.confusion_matrix(test_labels, pred))
#display(pd.crosstab(test_labels, list(pred), margins=True))

### Persist A Model
Save the model

In [ ]:
filepath = "player.mod"
model.save(filepath)

Load a saved model

In [ ]:
model = keras.models.load_model("player.mod")